In [1]:
# Deep Convolutional GANs

# Importing the libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

# Setting some hyperparameters
batchSize = 64 # We set the size of the batch.
imageSize = 64 # We set the size of the generated images (64x64).

In [2]:
# Creating the transformations
transform = transforms.Compose([transforms.Scale(imageSize), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]) # We create a list of transformations (scaling, tensor conversion, normalization) to apply to the input images.

# Loading the dataset
dataset = dset.CIFAR10(root = './data', download = True, transform = transform) # We download the training set in the ./data folder and we apply the previous transformations on each image.
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = True, num_workers = 2) # We use dataLoader to get the images of the training set batch by batch.

# Defining the weights_init function that takes as input a neural network m and that will initialize all its weights.
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

/opt/anaconda3/lib/python3.7/site-packages/torchvision/transforms/transforms.py:210: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +
0it [00:00, ?it/s]

100%|█████████▉| 170254336/170498071 [02:05<00:00, 1223235.11it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


170500096it [02:20, 1223235.11it/s]                               

In [4]:
class G(nn.Module):
    
    def __init__(self):
        super(G,self).__init__()
        self.main = nn.Sequential(
        nn.ConvTranspose2d(100,512,4,1,0,bias = False),
        nn.BatchNorm2d(512),
        nn.ReLU(True),
        nn.ConvTranspose2d(512,256,4,2,1,bias=False),
        nn.BatchNorm2d(256),
        nn.ReLU(True),
        nn.ConvTranspose2d(256,128,4,2,1,bias=False),   
        nn.BatchNorm2d(128),
        nn.ReLU(True),
        nn.ConvTranspose2d(128,64,4,2,1,bias=False),   
        nn.BatchNorm2d(64),
        nn.ReLU(True),
        nn.ConvTranspose2d(64,3,4,2,1,bias=False),   
        nn.Tanh()
        )
    
    def forward(self,input):
        output = self.main(input)
        return output

In [6]:
netG = G()
netG.apply(weights_init)

G(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [7]:
class D(nn.Module):
    
    def __init__(self):
        super(D,self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias = False),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1, bias = False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(512, 1, 4, 1, 0, bias = False),
            nn.Sigmoid()
            )
        
    def forward(self,input):
        output = self.main(input)
        return output.view(-1)
    

In [8]:
netD = D()
netD.apply(weights_init)

D(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

Training the Discriminator

In [9]:
criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(),lr=0.0002,betas=(0.5,0.999))
optimizerG = optim.Adam(netG.parameters(),lr=0.0002,betas=(0.5,0.999))

In [ ]:
for epoch in range(25):
    for i, data in enumerate(dataloader,0):
        netD.zero_grad()    #Updating the weights of the Discriminator
        real, _ = data      #Training the Discriminator with the real images
        input = Variable(real)
        target = Variable(torch.ones(input.size()[0]))
        output = netD(input)
        errD_real = criterion(output,target)
        
        noise = Variable(torch.randn(input.size()[0],100,1,1))  #Training the Discriminator with the fake images
        fake = netG(noise)
        target = Variable(torch.zeros(input.size()[0]))
        output = netD(fake.detach())
        errD_fake = criterion(output,target)
        
        errD = errD_real + errD_fake
        
        errD.backward()
        optimizerD.step()
        
        netG.zero_grad()  #Updating weights of the neural network
        target = Variable(torch.ones(input.size()[0]))
        output = netD(fake)
        errG = criterion(output,target)
        errG.backward()
        optimizerG.step()
        
        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, 25, i, len(dataloader), errD.item(), errG.item()))
        if i % 100 == 0:
            vutils.save_image(real, '%s/real_samples.png' % "/Users/vishalsingh/Desktop/Data/ML/ComputerVision/KirilEremenko/Computer_Vision_A_Z_Template_Folder/Module 3 - GANs/Module 3 - GANs", normalize = True)
            fake = netG(noise)
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("/Users/vishalsingh/Desktop/Data/ML/ComputerVision/KirilEremenko/Computer_Vision_A_Z_Template_Folder/Module 3 - GANs/Module 3 - GANs", epoch), normalize = True)

[0/25][0/782] Loss_D: 1.3440 Loss_G: 7.4288
[0/25][1/782] Loss_D: 0.7411 Loss_G: 7.8295
[0/25][2/782] Loss_D: 0.9017 Loss_G: 8.5447
[0/25][3/782] Loss_D: 0.9890 Loss_G: 8.2272
[0/25][4/782] Loss_D: 1.0104 Loss_G: 10.0585
[0/25][5/782] Loss_D: 0.3367 Loss_G: 8.5207
[0/25][6/782] Loss_D: 0.8483 Loss_G: 11.4859
[0/25][7/782] Loss_D: 0.4232 Loss_G: 8.4744
[0/25][8/782] Loss_D: 1.3623 Loss_G: 13.5716
[0/25][9/782] Loss_D: 0.5810 Loss_G: 9.9661
[0/25][10/782] Loss_D: 0.8197 Loss_G: 12.0120
[0/25][11/782] Loss_D: 0.5015 Loss_G: 9.0681
[0/25][12/782] Loss_D: 1.1789 Loss_G: 15.7198
[0/25][13/782] Loss_D: 0.6098 Loss_G: 12.3165
[0/25][14/782] Loss_D: 0.5917 Loss_G: 11.1134
[0/25][15/782] Loss_D: 0.7160 Loss_G: 15.2336
[0/25][16/782] Loss_D: 0.3985 Loss_G: 11.7431
[0/25][17/782] Loss_D: 0.3899 Loss_G: 10.1914
[0/25][18/782] Loss_D: 0.7470 Loss_G: 16.0262
[0/25][19/782] Loss_D: 0.4703 Loss_G: 12.9619
[0/25][20/782] Loss_D: 0.1826 Loss_G: 7.5673
[0/25][21/782] Loss_D: 2.8654 Loss_G: 20.9725
[0/25][

[0/25][179/782] Loss_D: 0.4253 Loss_G: 3.4374
[0/25][180/782] Loss_D: 0.9845 Loss_G: 12.4519
[0/25][181/782] Loss_D: 2.3035 Loss_G: 6.4247
[0/25][182/782] Loss_D: 0.1461 Loss_G: 3.6550
[0/25][183/782] Loss_D: 1.3695 Loss_G: 11.4226
[0/25][184/782] Loss_D: 1.8508 Loss_G: 7.2578
[0/25][185/782] Loss_D: 0.1179 Loss_G: 4.3682
[0/25][186/782] Loss_D: 0.7443 Loss_G: 9.3583
[0/25][187/782] Loss_D: 0.4757 Loss_G: 7.0337
[0/25][188/782] Loss_D: 0.2185 Loss_G: 4.8008
[0/25][189/782] Loss_D: 0.4457 Loss_G: 6.8602
[0/25][190/782] Loss_D: 0.4282 Loss_G: 4.8986
[0/25][191/782] Loss_D: 0.2725 Loss_G: 5.8286
[0/25][192/782] Loss_D: 0.3205 Loss_G: 6.1888
[0/25][193/782] Loss_D: 0.1434 Loss_G: 6.4549
[0/25][194/782] Loss_D: 0.5850 Loss_G: 5.0285
[0/25][195/782] Loss_D: 0.4526 Loss_G: 11.2013
[0/25][196/782] Loss_D: 0.3995 Loss_G: 7.9488
[0/25][197/782] Loss_D: 0.1390 Loss_G: 4.4186
[0/25][198/782] Loss_D: 0.7813 Loss_G: 11.6971
[0/25][199/782] Loss_D: 0.9613 Loss_G: 7.8002
[0/25][200/782] Loss_D: 0.2770

[0/25][357/782] Loss_D: 0.2730 Loss_G: 4.3689
[0/25][358/782] Loss_D: 0.2781 Loss_G: 7.9699
[0/25][359/782] Loss_D: 0.4295 Loss_G: 4.9594
[0/25][360/782] Loss_D: 0.2986 Loss_G: 6.1062
[0/25][361/782] Loss_D: 0.2646 Loss_G: 4.6238
[0/25][362/782] Loss_D: 0.1319 Loss_G: 5.5355
[0/25][363/782] Loss_D: 0.1181 Loss_G: 5.5111
[0/25][364/782] Loss_D: 0.2548 Loss_G: 4.4055
[0/25][365/782] Loss_D: 0.3249 Loss_G: 7.1618
[0/25][366/782] Loss_D: 0.3146 Loss_G: 4.0051
[0/25][367/782] Loss_D: 0.4212 Loss_G: 7.0757
[0/25][368/782] Loss_D: 0.2828 Loss_G: 4.9688
[0/25][369/782] Loss_D: 0.3476 Loss_G: 7.7509
[0/25][370/782] Loss_D: 0.2152 Loss_G: 5.7035
[0/25][371/782] Loss_D: 0.2557 Loss_G: 5.3874
[0/25][372/782] Loss_D: 0.2411 Loss_G: 6.0421
[0/25][373/782] Loss_D: 0.7026 Loss_G: 0.9669
[0/25][374/782] Loss_D: 2.0882 Loss_G: 15.8147
[0/25][375/782] Loss_D: 3.9830 Loss_G: 9.4900
[0/25][376/782] Loss_D: 0.3231 Loss_G: 5.4199
[0/25][377/782] Loss_D: 1.4365 Loss_G: 5.4661
[0/25][378/782] Loss_D: 0.9610 Lo

[0/25][535/782] Loss_D: 0.2057 Loss_G: 4.6236
[0/25][536/782] Loss_D: 0.2835 Loss_G: 4.3422
[0/25][537/782] Loss_D: 0.2799 Loss_G: 6.3989
[0/25][538/782] Loss_D: 0.1620 Loss_G: 6.3052
[0/25][539/782] Loss_D: 0.1148 Loss_G: 4.4559
[0/25][540/782] Loss_D: 0.2193 Loss_G: 5.4018
[0/25][541/782] Loss_D: 0.2051 Loss_G: 4.6353
[0/25][542/782] Loss_D: 0.1424 Loss_G: 4.6438
[0/25][543/782] Loss_D: 0.6276 Loss_G: 6.3534
[0/25][544/782] Loss_D: 0.4664 Loss_G: 4.1599
[0/25][545/782] Loss_D: 0.4961 Loss_G: 7.0047
[0/25][546/782] Loss_D: 0.4539 Loss_G: 5.1596
[0/25][547/782] Loss_D: 0.2962 Loss_G: 5.0169
[0/25][548/782] Loss_D: 0.2751 Loss_G: 6.0295
[0/25][549/782] Loss_D: 0.1363 Loss_G: 5.7649
[0/25][550/782] Loss_D: 0.2844 Loss_G: 6.2926
[0/25][551/782] Loss_D: 0.1605 Loss_G: 5.7823
[0/25][552/782] Loss_D: 0.1357 Loss_G: 4.9538
[0/25][553/782] Loss_D: 0.4266 Loss_G: 9.2649
[0/25][554/782] Loss_D: 0.3040 Loss_G: 7.2221
[0/25][555/782] Loss_D: 0.1108 Loss_G: 4.1928
[0/25][556/782] Loss_D: 0.3759 Los

[0/25][713/782] Loss_D: 0.3768 Loss_G: 4.2947
[0/25][714/782] Loss_D: 0.4436 Loss_G: 4.6819
[0/25][715/782] Loss_D: 0.2732 Loss_G: 4.3443
[0/25][716/782] Loss_D: 0.6236 Loss_G: 7.6169
[0/25][717/782] Loss_D: 0.5619 Loss_G: 3.9060
[0/25][718/782] Loss_D: 0.5697 Loss_G: 5.7930
[0/25][719/782] Loss_D: 0.7381 Loss_G: 5.2585
[0/25][720/782] Loss_D: 0.3269 Loss_G: 4.3693
[0/25][721/782] Loss_D: 0.3691 Loss_G: 6.5324
[0/25][722/782] Loss_D: 0.0841 Loss_G: 6.6565
[0/25][723/782] Loss_D: 0.2299 Loss_G: 4.6234
[0/25][724/782] Loss_D: 0.3273 Loss_G: 6.0279
[0/25][725/782] Loss_D: 0.2040 Loss_G: 5.3223
[0/25][726/782] Loss_D: 0.2980 Loss_G: 7.0663
[0/25][727/782] Loss_D: 0.3458 Loss_G: 4.2558
[0/25][728/782] Loss_D: 0.4164 Loss_G: 8.3841
[0/25][729/782] Loss_D: 0.3989 Loss_G: 4.7927
[0/25][730/782] Loss_D: 0.3908 Loss_G: 7.8424
[0/25][731/782] Loss_D: 0.4097 Loss_G: 4.8214
[0/25][732/782] Loss_D: 0.2618 Loss_G: 5.4162
[0/25][733/782] Loss_D: 0.2732 Loss_G: 5.6725
[0/25][734/782] Loss_D: 0.2968 Los

...

[1/25][291/782] Loss_D: 0.3876 Loss_G: 4.6757
[1/25][292/782] Loss_D: 0.3611 Loss_G: 3.1853
[1/25][293/782] Loss_D: 0.5956 Loss_G: 5.5594
[1/25][294/782] Loss_D: 0.6415 Loss_G: 2.4907
[1/25][295/782] Loss_D: 0.8133 Loss_G: 7.1940
[1/25][296/782] Loss_D: 0.6737 Loss_G: 3.8503
[1/25][297/782] Loss_D: 0.3835 Loss_G: 2.3782
[1/25][298/782] Loss_D: 0.9520 Loss_G: 8.0191
[1/25][299/782] Loss_D: 0.6737 Loss_G: 4.9072
[1/25][300/782] Loss_D: 0.3420 Loss_G: 2.3250
[1/25][301/782] Loss_D: 0.9278 Loss_G: 7.5533
[1/25][302/782] Loss_D: 0.5647 Loss_G: 4.7813
[1/25][303/782] Loss_D: 0.6588 Loss_G: 3.2321
[1/25][304/782] Loss_D: 1.0027 Loss_G: 6.0408
[1/25][305/782] Loss_D: 0.9430 Loss_G: 3.1479
[1/25][306/782] Loss_D: 0.7131 Loss_G: 4.1451
[1/25][307/782] Loss_D: 0.3220 Loss_G: 4.6047
[1/25][308/782] Loss_D: 0.3009 Loss_G: 3.6634
[1/25][309/782] Loss_D: 0.7005 Loss_G: 4.7286
[1/25][310/782] Loss_D: 0.4040 Loss_G: 3.7870
[1/25][311/782] Loss_D: 0.6336 Loss_G: 3.7419
[1/25][312/782] Loss_D: 0.8665 Los

[1/25][470/782] Loss_D: 2.4539 Loss_G: 3.4163
[1/25][471/782] Loss_D: 0.3968 Loss_G: 2.9679
[1/25][472/782] Loss_D: 0.7761 Loss_G: 7.0695
[1/25][473/782] Loss_D: 1.3592 Loss_G: 3.1276
[1/25][474/782] Loss_D: 0.6089 Loss_G: 4.0121
[1/25][475/782] Loss_D: 0.4738 Loss_G: 5.8071
[1/25][476/782] Loss_D: 0.6377 Loss_G: 3.3640
[1/25][477/782] Loss_D: 0.4830 Loss_G: 3.7554
[1/25][478/782] Loss_D: 0.5063 Loss_G: 4.2813
[1/25][479/782] Loss_D: 0.5390 Loss_G: 3.2729
[1/25][480/782] Loss_D: 1.2212 Loss_G: 2.9807
[1/25][481/782] Loss_D: 0.7948 Loss_G: 4.7227
[1/25][482/782] Loss_D: 1.3820 Loss_G: 2.0950
[1/25][483/782] Loss_D: 0.9586 Loss_G: 6.3572
[1/25][484/782] Loss_D: 0.4972 Loss_G: 4.3321
[1/25][485/782] Loss_D: 0.4646 Loss_G: 2.9343
[1/25][486/782] Loss_D: 0.4710 Loss_G: 4.5146
[1/25][487/782] Loss_D: 0.7697 Loss_G: 4.5574
[1/25][488/782] Loss_D: 0.3791 Loss_G: 3.3593
[1/25][489/782] Loss_D: 0.6279 Loss_G: 4.5674
[1/25][490/782] Loss_D: 0.2858 Loss_G: 4.2418
[1/25][491/782] Loss_D: 0.3038 Los

[1/25][649/782] Loss_D: 0.5066 Loss_G: 3.0829
[1/25][650/782] Loss_D: 0.3503 Loss_G: 4.0595
[1/25][651/782] Loss_D: 0.3075 Loss_G: 3.7590
[1/25][652/782] Loss_D: 0.2419 Loss_G: 3.7718
[1/25][653/782] Loss_D: 0.2941 Loss_G: 3.6726
[1/25][654/782] Loss_D: 0.2801 Loss_G: 3.8477
[1/25][655/782] Loss_D: 0.5424 Loss_G: 4.3421
[1/25][656/782] Loss_D: 0.6543 Loss_G: 1.9733
[1/25][657/782] Loss_D: 0.9858 Loss_G: 7.1022
[1/25][658/782] Loss_D: 1.3924 Loss_G: 2.5377
[1/25][659/782] Loss_D: 0.4385 Loss_G: 3.1463
[1/25][660/782] Loss_D: 0.2922 Loss_G: 4.8956
[1/25][661/782] Loss_D: 0.6293 Loss_G: 2.4313
[1/25][662/782] Loss_D: 0.3712 Loss_G: 4.2785
[1/25][663/782] Loss_D: 0.3310 Loss_G: 4.5573
[1/25][664/782] Loss_D: 0.2613 Loss_G: 3.5061
[1/25][665/782] Loss_D: 0.3097 Loss_G: 3.4699
[1/25][666/782] Loss_D: 0.4544 Loss_G: 3.3664
[1/25][667/782] Loss_D: 0.5189 Loss_G: 3.0525
[1/25][668/782] Loss_D: 0.3434 Loss_G: 5.1107
[1/25][669/782] Loss_D: 0.4725 Loss_G: 2.8052
[1/25][670/782] Loss_D: 0.4479 Los

[2/25][47/782] Loss_D: 0.6617 Loss_G: 2.3845
[2/25][48/782] Loss_D: 0.4814 Loss_G: 3.1543
[2/25][49/782] Loss_D: 0.4423 Loss_G: 2.8674
[2/25][50/782] Loss_D: 0.4940 Loss_G: 4.0793
[2/25][51/782] Loss_D: 0.5086 Loss_G: 2.7906
[2/25][52/782] Loss_D: 0.3963 Loss_G: 2.6660
[2/25][53/782] Loss_D: 0.4046 Loss_G: 4.2817
[2/25][54/782] Loss_D: 0.5528 Loss_G: 2.1228
[2/25][55/782] Loss_D: 0.3987 Loss_G: 3.4442
[2/25][56/782] Loss_D: 0.4833 Loss_G: 4.9435
[2/25][57/782] Loss_D: 0.5171 Loss_G: 2.0322
[2/25][58/782] Loss_D: 0.6158 Loss_G: 5.2600
[2/25][59/782] Loss_D: 0.7343 Loss_G: 1.1274
[2/25][60/782] Loss_D: 1.6110 Loss_G: 7.7522
[2/25][61/782] Loss_D: 1.6828 Loss_G: 1.4264
[2/25][62/782] Loss_D: 1.1409 Loss_G: 5.6492
[2/25][63/782] Loss_D: 0.8049 Loss_G: 2.0953
[2/25][64/782] Loss_D: 0.6605 Loss_G: 3.3802
[2/25][65/782] Loss_D: 0.4625 Loss_G: 4.8685
[2/25][66/782] Loss_D: 0.7170 Loss_G: 2.0406
[2/25][67/782] Loss_D: 0.9258 Loss_G: 5.0484
[2/25][68/782] Loss_D: 0.7001 Loss_G: 2.5831
[2/25][69/

[2/25][227/782] Loss_D: 2.1745 Loss_G: 1.9912
[2/25][228/782] Loss_D: 0.7058 Loss_G: 1.4572
[2/25][229/782] Loss_D: 0.9690 Loss_G: 5.8747
[2/25][230/782] Loss_D: 1.1655 Loss_G: 1.6145
[2/25][231/782] Loss_D: 0.9261 Loss_G: 3.1554
[2/25][232/782] Loss_D: 0.9253 Loss_G: 2.7417
[2/25][233/782] Loss_D: 1.1893 Loss_G: 1.5713
[2/25][234/782] Loss_D: 1.3351 Loss_G: 4.6588
[2/25][235/782] Loss_D: 1.2107 Loss_G: 1.8153
[2/25][236/782] Loss_D: 0.5195 Loss_G: 3.3767
[2/25][237/782] Loss_D: 0.5392 Loss_G: 3.1854
[2/25][238/782] Loss_D: 0.8677 Loss_G: 1.6368
[2/25][239/782] Loss_D: 0.8374 Loss_G: 3.5446
[2/25][240/782] Loss_D: 0.5525 Loss_G: 2.6668
[2/25][241/782] Loss_D: 0.4436 Loss_G: 2.5849
[2/25][242/782] Loss_D: 0.6340 Loss_G: 4.2506
[2/25][243/782] Loss_D: 0.7567 Loss_G: 1.9482
[2/25][244/782] Loss_D: 0.7249 Loss_G: 2.9524
[2/25][245/782] Loss_D: 0.4420 Loss_G: 3.3357
[2/25][246/782] Loss_D: 0.8015 Loss_G: 2.5074
[2/25][247/782] Loss_D: 0.5456 Loss_G: 3.2680
[2/25][248/782] Loss_D: 0.8491 Los